# **Installing Necessary Libraries**

In [1]:
!pip install -qqq plotly sentence-transformers tiktoken langchain openai==0.28 PyPDF2 qdrant_client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# **Importing Libraries and Setting Up Environment**

In [4]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
import qdrant_client as qc
import qdrant_client.http.models as qmodels
from qdrant_client.http.models import *
#from qdrant_client import QdrantClient,models
#from qdrant_client.http.models import PointStruct
import os
import uuid
import openai

# **Defining File Path for Data Storage**

In [34]:
file_path = "/content/data/"

# **Function to Extract Text from PDF Files**

In [36]:
def get_pdf_data(file_path, num_pages = 1):
    reader = PdfReader(file_path)
    full_doc_text = ""
    pages = reader.pages
    num_pages = len(pages)

    try:
        for page in range(num_pages):
            current_page = reader.pages[page]
            text = current_page.extract_text()
            full_doc_text += text
    except:
        print("Error reading file")
    finally:
        return full_doc_text

# **Function to Split Text into Manageable Chunks**

In [15]:
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)

    return chunks


# **Initializing the Sentence Transformer Model**

In [16]:
model = SentenceTransformer('all-mpnet-base-v2')

In [17]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


# **Creating a Qdrant Connection and Managing Collections**

In [18]:
import requests
import tempfile


In [19]:
record = 0


from qdrant_client import QdrantClient

connection = QdrantClient(
    url="https://40535b3f-d117-49dd-8bdc-6fb826edc0c6.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="-7PSR7d60KlZ5SlCaHThUM-yST1aGm24lPP0xs9uIkqYn8o85-cRAw",
)

In [64]:
collection_name = "question-answering-v1"

In [65]:
collections = connection.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
  connection.create_collection(
    collection_name = collection_name,
    vectors_config = models.VectorParams(size=768, distance = models.Distance.COSINE)
)

collections=[CollectionDescription(name='extractive-question-answering'), CollectionDescription(name='question-answering')]


In [66]:
print("Create collection response:", connection)

Create collection response: <qdrant_client.qdrant_client.QdrantClient object at 0x784ae12d6800>


In [67]:
info = connection.get_collection(collection_name = collection_name)


In [68]:
print("Collection info:", info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}


In [69]:
for get_info in info:
  print(get_info)

('status', <CollectionStatus.GREEN: 'green'>)
('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>)
('vectors_count', 0)
('indexed_vectors_count', 0)
('points_count', 0)
('segments_count', 2)
('config', CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None))
('payloa

In [70]:
FILE_PATH = "/content/data/"
COLLECTION_NAME = collection_name

# **Upserting Chunks**

In [71]:
FILES = os.listdir(FILE_PATH)
FILES_FULL_PATH = [FILE_PATH + file for file in FILES]
for filename in FILES_FULL_PATH:
    print(f'Processing file: {filename}')
    full_doc_text = get_pdf_data(filename)
    print(f'Full doc text length: {len(full_doc_text)}')
    payloads = []
    li_id = []
    corpus = []
    Lines =get_chunks(full_doc_text,500)
    for token in Lines:
        corpus.append(token)
        payloads.append({"token":token,
                         "filename": os.path.basename(filename),
                           "type":"pdf"})
        li_id.append(str(uuid.uuid4()))
    embeddings_all = model.encode(corpus, convert_to_tensor=True)
    print(f'Full embeddings length: {len(embeddings_all)}')

    CHUNK_SIZE = 100
    for i in range(0, len(embeddings_all), CHUNK_SIZE):
        if(i+CHUNK_SIZE > len(embeddings_all) -1):
            new_chunk = len(embeddings_all) -1
        else:
            new_chunk = i+CHUNK_SIZE -1
        print("Inserting chunk", i , "to", new_chunk)
        connection.upsert(
            collection_name=COLLECTION_NAME,
            points=qmodels.Batch(
                ids = li_id[i:new_chunk],
                vectors=embeddings_all[i:new_chunk].tolist(),
                payloads=payloads[i:new_chunk]
            ),
        )

Processing file: /content/data/dermatologia.pdf
Full doc text length: 17780
Full embeddings length: 41
Inserting chunk 0 to 40
